## **Round Robin (RR) Code**

In [1]:
from collections import deque

sample_processes=[ #Sample dataset with pid, arrival time, burst time, and priority (can add more PIDs or the values can be changed)
    {"pid": "P1", "arrival": 0, "burst": 10, "priority": 4},
    {"pid": "P2", "arrival": 1, "burst": 6, "priority": 3},
    {"pid": "P3", "arrival": 2, "burst": 11, "priority": 5},
    {"pid": "P4", "arrival": 3, "burst": 7, "priority": 4}
]

def print_schedule(results): #Function to print scheduling results in a table
    print(f"{'PID':<5} {'Start':<6} {'Finish':<6} {'Waiting':<7} {'Turnaround':<10}") #Print the table with fixed column widths
    for r in results: #Loop through each result entry (each process)
        print(f"{r['pid']:<5} {r['start']:<6} {r['finish']:<6} "
              f"{r['waiting']:<7} {r['turnaround']:<10}")

def round_robin(sample_processes, quantum): #RR scheduling with time quantum
    queue = deque() #Queue to hold ready processes (First In First Out)
    sample_processes = [{**p, "remaining": p["burst"]} for p in sample_processes] #Add "remaining" burst time for preemption
    time = 0 #Current simulation time
    results = {} #Stores results by PID

    sample_processes = sorted(sample_processes, key=lambda x: x["arrival"]) #Sort by arrival time for processing
    
    i = 0
    while i < len(sample_processes) or queue: #Continue while processes remain or queue not empty
        while i < len(sample_processes) and sample_processes[i]["arrival"] <= time:
            queue.append(sample_processes[i]) #Add arriving process to ready queue
            i += 1

        if not queue: #If queue empty, jump time to next arrival
            time = sample_processes[i]["arrival"]
            continue

        p = queue.popleft() #Get next process in RR order

        if p["pid"] not in results: #If first time running the process, record its first start time
            results[p["pid"]] = {"pid": p["pid"], "start": time}

        run_time = min(quantum, p["remaining"]) #Execute up to quantum or remaining time
        p["remaining"] -= run_time #Reduce remaining burst
        time += run_time #Advance simulation time

        while i < len(sample_processes) and sample_processes[i]["arrival"] <= time: #Add any newly arrived processes during this time slice
            queue.append(sample_processes[i])
            i += 1

        if p["remaining"] > 0: # If process still not finished, requeue it to get another time slice
            queue.append(p)
        else: # If finished, record completion time
            results[p["pid"]]["finish"] = time
            results[p["pid"]]["waiting"] = (results[p["pid"]]["finish"] - p["arrival"] - p["burst"]) #Waiting = finish - arrival - burst
            results[p["pid"]]["turnaround"] = (results[p["pid"]]["finish"] - p["arrival"]) #Turnaround = finish - arrival

    return list(results.values()) #Convert dict to list

print("Round Robin (q=3):")
print_schedule(round_robin(sample_processes, quantum=3))

Round Robin (q=3):
PID   Start  Finish Waiting Turnaround
P1    0      32     22      32        
P2    3      18     11      17        
P3    6      34     21      32        
P4    9      31     21      28        


## **Gantt Chart & Averages Code**

In [3]:
def make_gantt_chart(results):
    results = sorted(results, key=lambda x: x["start"]) #Sort tasks by start time for correct chart order

    chart = "" #For bar representation
    timeline = "" #For timestamp markers

    for r in results: #Build bars and timeline for each process
        length = r["finish"] - r["start"] #Duration of process execution
        bar = "-" * length
        chart += f"|{bar}"
        timeline += f"{r['start']}{' ' * length}" #Add start time and spacing

    timeline += f"{results[-1]['finish']}" #Add final ending time

    print("\nGantt Chart:")
    print(chart + "|")

    label_line = ""
    for r in results: #Create a line with PID labels centered in each bar
        length = r["finish"] - r["start"]
        label_line += "|" + r["pid"].center(length, "-") #Center PID inside the block
    print(label_line + "|")

    print(timeline) #Print timeline with time markers

def compute_averages(results): #Compute average waiting & turnaround times
    total_wait = sum(r["waiting"] for r in results) #Sum of all waiting times
    total_turn = sum(r["turnaround"] for r in results) #Sum of all turnaround times
    n = len(results) #Number of processes
    return {"avg_waiting": total_wait / n, "avg_turnaround": total_turn / n} # Average waiting and turnaround time

results = round_robin(sample_processes, quantum=3)
make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])


Gantt Chart:
|--------------------------------|---------------|----------------------------|----------------------|
|---------------P1---------------|-------P2------|-------------P3-------------|----------P4----------|
0                                3               6                            9                      31

Average Waiting Time = 18.75
Average Turnaround Time = 27.25
